## 使用 GPTQ 量化模型 2.7B

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

#### 逐层量化

In [3]:
# !pip install optimum
# !pip install auto-gptq

In [4]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

/opt/conda/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/opt/conda/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/opt/conda/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/transformers/modeling_utils.py:5005: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [5]:
!nvidia-smi -i 5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Aug  6 01:29:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   5  NVIDIA GeForce RTX 4090        Off |   00000000:AE:00.0 Off |                  Off |
| 44%   39C    P2             61W /  450W |    7889MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 检查量化模型正确性

In [6]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': {},
 '_buffers': {'qweight': tensor([[ 1766754698, -1249142373,  1183631034,  ..., -2038658921,
           -2037544795, -1956877206],
          [ 1772710025,  1739893370, -1500087466,  ...,  2021033895,
            -662329995,  1756019066],
          [ -895658394, -2007414633, -1951893913,  ..., -1429760649,
            -980833883,  1451914633],
          ...,
          [ 2025363833, -1412855115,  1539086490,  ...,  -342189415,
           -1737062521, -1950833303],
          [ 2023326856, -1432974698, -1788251003,  ..., -1161734024,
            2043176297,  1449571741],
          [ 1769506966, -2021291654, -1182103157,  ...,  1999993942,
            1753778600,  1970816134]], device='cuda:0', dtype=torch.int32),
  'qzeros': tensor([[2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 20043180

In [7]:
#  device(type='cuda', index=0)，因为设置了 环境变量的cuda的可见性，所以这里的 index=0 是 卡5

In [8]:
model_name_or_path

'facebook/opt-2.7b'

In [9]:
model_name_local = '../model/wyd/opt-2.7b-gptq'
# 保存模型权重
quant_model.save_pretrained(model_name_local)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around.
I'm still around, just not as much as I used to be.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

In [11]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/transformers/modeling_utils.py:5005: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


相比使用默认数据集，未经精心准备的自定义数据集会明显降低模型性能

In [12]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.



# Homework： 使用 GPTQ 量化模型 6.7B

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

In [14]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

# 检查模型合规性

In [15]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': {},
 '_buffers': {'qweight': tensor([[-1533367701,  2022282410, -1182414405,  ..., -1537760937,
             968190105, -1218869078],
          [-1973762678, -1990888091,  1451669112,  ...,  1767327094,
            1485273242, -1769109111],
          [ -890521657,  1705355194,  2042256023,  ...,  1401453177,
            -963081656, -1212573545],
          ...,
          [-1192544404,   697191045,  1432856694,  ...,  1967820506,
           -1482119368, -1787262823],
          [-1736931943, -1753576812,  2027985786,  ..., -1757504693,
            2090308806, -1987483739],
          [ 1549191317,  1151064006, -1735993498,  ..., -1317428394,
           -1182375288, -1199925157]], device='cuda:0', dtype=torch.int32),
  'qzeros': tensor([[2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 2004318071, 2004318071,  ..., 2004318071, 2004318071,
           2004318071],
          [2004318071, 20043180

In [16]:
model_name_local = '../model/wyd/opt-6.7b-gptq'
# 保存模型权重
quant_model.save_pretrained(model_name_local)

# GPTQ “opt-6.7b-gptq” 量化后模型测试

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to hear you're doing better.
Thank you! Merry Christmas to you too!


# Transformers 模型量化技术：AWQ

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
from transformers import pipeline

model_path = "facebook/opt-125m"



In [3]:
# 使用 GPU 加载原始的 OPT-125m 模型
generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
!nvidia-smi -i 5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Aug  7 01:55:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   5  NVIDIA GeForce RTX 4090        Off |   00000000:AE:00.0 Off |                  Off |
| 44%   32C    P2             22W /  450W |     921MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
generator("The woman worked as a")


[{'generated_text': 'The woman worked as a security guard with the Department of Homeland Security and she is now married to U'},
 {'generated_text': 'The woman worked as a maid for K.U.L. but moved out. So then she'},
 {'generated_text': "The woman worked as a security guard. She was there most of her life and was her family's"}]

In [6]:
generator("The man worked as a")


[{'generated_text': 'The man worked as a nurse for 40 years, the man was not a professional. We call them'},
 {'generated_text': 'The man worked as a busboy in a bus stop.  It used to be called "the'},
 {'generated_text': 'The man worked as a server while teaching us how to create an online business. This is one of'}]

In [7]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path = "models/opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 12/12 [00:39<00:00,  3.28s/it]


In [9]:
!nvidia-smi -i 5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Aug  7 01:57:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   5  NVIDIA GeForce RTX 4090        Off |   00000000:AE:00.0 Off |                  Off |
| 44%   34C    P8             11W /  450W |    1077MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

# Transformers 兼容性配置


In [14]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [15]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('.models/opt-125m-awq/tokenizer_config.json',
 '.models/opt-125m-awq/special_tokens_map.json',
 '.models/opt-125m-awq/vocab.json',
 '.models/opt-125m-awq/merges.txt',
 '.models/opt-125m-awq/added_tokens.json',
 '.models/opt-125m-awq/tokenizer.json')

# 使用 GPU 加载量化模型

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [17]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [18]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to see people up this got some up, thank thank to
It's do, thanks to, thank you for
thank you, thank t

Gotta, thanks m

A Merry, thank thank god you

a Merry, thank

-Merry
-Berry
 -

C


In [19]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a pimp in the office also makes as the pimp in the office is the pimp in the office has the pimp in the office, we made the pimp for the pimp but the pimp with the pimp is the penis also

fak the penis in the pimp in the pen p


# Homework：使用 AWQ 算法量化 Facebook OPT-2.7B 模型

In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [23]:
from transformers import pipeline

model_path = "facebook/opt-2.7b"



In [24]:
# 使用 GPU 加载原始的 opt-2.7b 模型
generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

In [25]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 12/12 [00:46<00:00,  3.88s/it]


In [26]:
!nvidia-smi -i 5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Aug  7 02:11:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   5  NVIDIA GeForce RTX 4090        Off |   00000000:AE:00.0 Off |                  Off |
| 44%   38C    P2             65W /  450W |   10983MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

In [28]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [29]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('../models/facebook/opt-2.7b-awq/tokenizer_config.json',
 '../models/facebook/opt-2.7b-awq/special_tokens_map.json',
 '../models/facebook/opt-2.7b-awq/vocab.json',
 '../models/facebook/opt-2.7b-awq/merges.txt',
 '../models/facebook/opt-2.7b-awq/added_tokens.json',
 '../models/facebook/opt-2.7b-awq/tokenizer.json')

# 使用 GPU 加载量化模型

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [31]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [32]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to see you in here, here, I am here, I am here again, I am not too here

(I'm done
; I was
im too too,

( I'm in do
) here, I am home

(I'm here
(I'm my,
I


In [33]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a woman who stole it for a girl she was the woman she didn?

The guy you you don?
The

e
t you did

the guy
that u



t

y

o

i
f
y

i

i


